In [1]:
VRS_LATITUDE = 25.3176
VRS_LONGITUDE = 82.9739
print('The geograpical coordinates of Varanasi are {}, {}.'.format(VRS_LATITUDE, VRS_LONGITUDE))

The geograpical coordinates of Varanasi are 25.3176, 82.9739.


In [2]:
FOURSQUARE_CLIENT_ID = 'GMUUC2GUMFOORGUE1X1RP5TDHGBFHG05QV0RBE5G3DIWIMVG'
FOURSQUARE_CLIENT_SECRET = 'AKKTWEFY02L5UQCS0P2KKNHOLPQQT1AZWPQMIWSO0AMJ2EI0'
RADIUS = 6000 # 4 Km
NO_OF_VENUES = 100
VERSION = '20190612' # Current date

In [3]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize
import requests

pd.set_option('display.max_rows', None)

offset = 0
total_venues = 0
foursquare_venues = pd.DataFrame(columns = ['name', 'categories', 'lat', 'lng'])

while (True):
    url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
           '&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}').format(FOURSQUARE_CLIENT_ID, 
                                                                        FOURSQUARE_CLIENT_SECRET, 
                                                                        VERSION, 
                                                                        VRS_LATITUDE, 
                                                                        VRS_LONGITUDE,
                                                                        RADIUS,
                                                                        NO_OF_VENUES,
                                                                        offset)
    result = requests.get(url).json()
    venues_fetched = len(result['response']['groups'][0]['items'])
    total_venues = total_venues + venues_fetched
    print("Total {} venues fetched within a total radius of {} Km".format(venues_fetched, RADIUS/1000))

    venues = result['response']['groups'][0]['items']
    venues = json_normalize(venues)

    # Filter the columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    venues = venues.loc[:, filtered_columns]

    # Filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis = 1)

    # Clean all column names
    venues.columns = [col.split(".")[-1] for col in venues.columns]
    foursquare_venues = pd.concat([foursquare_venues, venues], axis = 0, sort = False)
    
    if (venues_fetched < 100):
        break
    else:
        offset = offset + 100

foursquare_venues = foursquare_venues.reset_index(drop = True)
print("\nTotal {} venues fetched".format(total_venues))

Total 41 venues fetched within a total radius of 6.0 Km

Total 41 venues fetched


In [5]:
headers = {'user-key': 'b3a1c26ed20422bfaae7ada558744e1f'}
venues_information = []

for index, row in foursquare_venues.iterrows():
    print("Fetching data for venue: {}".format(index + 1))
    venue = []
    url = ('https://developers.zomato.com/api/v2.1/search?q={}' + 
          '&start=0&count=1&lat={}&lon={}&sort=real_distance').format(row['name'], row['lat'], row['lng'])
    result = requests.get(url, headers = headers).json()
    if (len(result['restaurants']) > 0):
        venue.append(result['restaurants'][0]['restaurant']['name'])
        venue.append(result['restaurants'][0]['restaurant']['location']['latitude'])
        venue.append(result['restaurants'][0]['restaurant']['location']['longitude'])
        venue.append(result['restaurants'][0]['restaurant']['average_cost_for_two'])
        venue.append(result['restaurants'][0]['restaurant']['price_range'])
        venue.append(result['restaurants'][0]['restaurant']['user_rating']['aggregate_rating'])
        venue.append(result['restaurants'][0]['restaurant']['location']['address'])
        venues_information.append(venue)
    else:
        venues_information.append(np.zeros(6))
    
zomato_venues = pd.DataFrame(venues_information, 
                                  columns = ['venue', 'latitude', 
                                             'longitude', 'price_for_two', 
                                             'price_range', 'rating', 'address'])

Fetching data for venue: 1
Fetching data for venue: 2
Fetching data for venue: 3
Fetching data for venue: 4
Fetching data for venue: 5
Fetching data for venue: 6
Fetching data for venue: 7
Fetching data for venue: 8
Fetching data for venue: 9
Fetching data for venue: 10
Fetching data for venue: 11
Fetching data for venue: 12
Fetching data for venue: 13
Fetching data for venue: 14
Fetching data for venue: 15
Fetching data for venue: 16
Fetching data for venue: 17
Fetching data for venue: 18
Fetching data for venue: 19
Fetching data for venue: 20
Fetching data for venue: 21
Fetching data for venue: 22
Fetching data for venue: 23
Fetching data for venue: 24
Fetching data for venue: 25
Fetching data for venue: 26
Fetching data for venue: 27
Fetching data for venue: 28
Fetching data for venue: 29
Fetching data for venue: 30
Fetching data for venue: 31
Fetching data for venue: 32
Fetching data for venue: 33
Fetching data for venue: 34
Fetching data for venue: 35
Fetching data for venue: 36
F

In [6]:
zomato_venues

,venue,latitude,longitude,price_for_two,price_range,rating,address
0,Hotel Dolphin International,25.2995300290,82.9995642040,500.0,3.0,3.8,"B 20/48, Near Jain Mandir, Bhelupur, Varanasi"
1,Punjab Grill House,25.3094922309,83.0045169219,300.0,2.0,2.9,"Near Church Crossing, Luxa Road, Godaulia, Var..."
2,Kashi Chat Bhandar,25.3093913007,83.0059475452,150.0,1.0,4.8,"D-37/49, Godaulia, Varanasi"
3,Dawat,25.3096328663,83.0053490773,1200.0,4.0,3.0,"Hotel Ganges Grand, Godowlia Crossing, Godauli..."
4,UBQ by Barbeque Nation,25.3356311000,82.9776407000,400.0,2.0,3.1,"JHV Mall, 4th, The Mall Rd, Near Ramada Hotel,..."
5,0,0,0,0.0,0.0,0,None
6,Dolphin Restaurant,25.3078649107,83.0099621415,700.0,3.0,3.7,"D 16/30 Manmandir Ghat, Dashwamedh Road, Varanasi"
7,The Siwon Lassi Shop,25.3114830000,83.0111220000,100.0,1.0,3.6,"Tripura Bhairwi Road, Nepali Kapda, Dashaswmed..."
8,Sushi Cafe & Continental Restaurant,25.3053200000,83.0084490000,400.0,2.0,3.4,"D-32, 13-14, Bangali Tola Road, Bangali Tola, ..."
9,Sol Bar - Hotel Surya,25.3387057134,82.9798902199,1500.0,4.0,3.6,"Hotel Surya, 20/51, A-5, The Mall Road, Varuna..."
